#### Sequential Feature Selection of PCs for Random Forest model using PV/GPH/EHF as input.

File duplicated on May 8, 2025.

File editted on June 4th. Updated to use new timeseries of data for temperature. Also attempting to remove necessity for multiple solvers by timeseries. 

This specific file is for the "real" analysis and will not have different PCs depending on leadtimes. 

I would like to identify the # of nodes that represent >80% of the variance for each feature. Then, I will do two models.  

FILE COPIED FROM REPO 1/5/2026 TO CONDUCT SEPARATION ON DATA AND EOF ANALYSIS BASED ON TRAIN/TEST SPLIT. 

In [1]:
# relevant import statements
import math
import pickle
import random

##just to stop the excess number of warnings
import warnings
warnings.filterwarnings("ignore")
from random import randint, sample, seed

# plotting related imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns  # statistical data visualization
import xarray as xr

from eofs.standard import Eof

The bulk of this file to start is copied from the previous EOF file/my EOF test file. 

##### First, I am going to pickle in the input data. I will then remove the seasonal climo from the dataset...

I had to actually download the data locally and upload it here to the H100 cluster in the ./Dissertation_Coding/data folder because I could not call it from my home directory. 

In [2]:
# load input data
infile = open("../processed_vertical_data/vertanom_u.p","rb",)
u_input = pickle.load(infile)  ##vertical U cross section
infile.close()

infile = open("../processed_vertical_data/vertanom_ehf.p","rb",)
ehf_input = pickle.load(infile)  ##ZMehf vertical cross section along longitudes
infile.close()

infile = open("../processed_vertical_data/vertanom_gph.p","rb",)
gph_input = pickle.load(infile)  ##ZMehf vertical cross section along longitudes
infile.close()

In [3]:
u_input.shape  ##62 years, october through march,

(62, 182, 37, 36)

In [4]:
gph_input.shape

(62, 182, 37, 180)

In [5]:
ehf_input.shape

(62, 182, 37, 180)

#### Great, so I have everything uploaded and reduced to daily anomalies with the seasonal climo removed. Fantastic, lol. 

##### Attempt at EOF analysis/PC decomp based on Zheng's example code. 

In [6]:
print('14 DAYS TRAIN')
##flatten array to just the dates relevant to my models. 
flat_u_14 = u_input[:52, 19:168, :, :].reshape((52 * 149, 37, 36))
print(flat_u_14.shape)

flat_EHF_14 = ehf_input[:52, 19:168, :, :].reshape((52 * 149, 37, 180))
print(flat_EHF_14.shape)

flat_GPH_14= gph_input[:52, 19:168, :, :].reshape((52 * 149, 37, 180))
print(flat_GPH_14.shape)

Usolver_14_train = Eof(flat_u_14)
EHFsolver_14_train = Eof(flat_EHF_14)
GPHsolver_14_train = Eof(flat_GPH_14)

14 DAYS TRAIN
(7748, 37, 36)
(7748, 37, 180)
(7748, 37, 180)


In [7]:
print('14 DAYS TEST')
##flatten array to just the dates relevant to my models. 
flat_u_14 = u_input[52:, 19:168, :, :].reshape((10 * 149, 37, 36))
print(flat_u_14.shape)

flat_EHF_14 = ehf_input[52:, 19:168, :, :].reshape((10 * 149, 37, 180))
print(flat_EHF_14.shape)

flat_GPH_14= gph_input[52:, 19:168, :, :].reshape((10 * 149, 37, 180))
print(flat_GPH_14.shape)

Usolver_14_test = Eof(flat_u_14)
EHFsolver_14_test = Eof(flat_EHF_14)
GPHsolver_14_test = Eof(flat_GPH_14)

14 DAYS TEST
(1490, 37, 36)
(1490, 37, 180)
(1490, 37, 180)


In [1]:
## Only the training solvers are saved out so that these PCs can be extrapolated onto the testing data.

In [8]:
pickle.dump(Usolver_14_train, open("Usolver_14_train.p", 'wb'))
pickle.dump(EHFsolver_14_train, open("EHFsolver_14_train.p", 'wb'))
pickle.dump(GPHsolver_14_train, open("GPHsolver_14_train.p", 'wb'))